In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
from  lib.advection import xr_vert_adv
from lib.thermo import liquid_water_temperature, get_dz

In [ ]:
stat = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc")
fields_3d = xr.open_mfdataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/*.nc")
fields_2d = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc")

# for some reason the time values are all scrambled up.
fields_2d = fields_2d.isel(time=fields_2d.time.values.argsort())
data = xr.merge((fields_2d, fields_3d), join='inner').isel(y=slice(24, 40))

rho = stat.RHO[0]
dz = get_dz(rho.z)
data  = data.assign(
    sl=liquid_water_temperature(data.TABS, data.QN, data.QP)
)

# Mass conservation in the coarse-grained fields

The coarse-grained horizontal velocities do not have to obey the divergence free condition, which leads to large errors when computing the advection form versus divergence form. The simplest way to verify that the equations do not have a good divergence free condition is to take a centered difference of the velocity field. The data are all collocated, so this is the only simple divergence operator. We could also use a spectral basis, but this is more complicated. I will compute the vertically averaged divergence to show that the data cannot be divergence free.
$$ w(z) =-\langle \delta \rangle =-\frac{1}{\rho_0} \int_0^z \rho_0 (u_x + u_y) dz  $$
To satisfy mass conservation, $w(H)$ must be zero, which is equivalent to saying that the vertical velocity vanishes at the upper boundary.


In [ ]:
div = data.U.centderiv('x') + data.V.centderiv('y')
w = -(div*rho*dz).cumsum('z').compute()/rho

If we plot this vertically averaged divergence compute numerically from our data at our favorite spatial location at z=6555, we get the following time series.

In [ ]:
plt.figure(figsize=(6,2), dpi=100)
w.isel(x=0, y=8, z=10).plot(label='divergence', alpha=1.0)
data.W.isel(x=0, y=8, z=15).plot(label='coarse grained', alpha=.5)
plt.legend()

This shows that the divergence based estimate of $W$ is substantially difference from $W$. In fact the situation is even worse, and mass is not conserved in the column. See the following time series.

In [ ]:
plt.figure(figsize=(6,2), dpi=100)
w.isel(x=0, y=8, z=-1).plot(label='coarse grained')
plt.legend()

This is also obvious in the z vs time plot of the divergence estimated vertical velocity:

In [ ]:
plt.figure(figsize=(6,2), dpi=100)
w.isel(x=0,y=8).T.plot()

The non-conservation of mass in the coarse-grained fields makes computing the source terms relatively sensitive to the method used. For instance, the flux form and advection form do not have the same answer. In other words,
$$ \nabla \cdot u f + \frac{1}{\rho_0}(uf\rho_0)_z \ne u\cdot\nabla f + w f_z .$$

We have found in our work so far that the advection form gives more reliable estimates of advection forcings, while the divergence form is very sensitive to numerical cancelation. I found the tendencies in flux form are 10 times larger than in advective form. This results in predicted time mean temperature tendencies of 100s of K/day, a very unrealistic results.

To fix this, we could consider first projecting the coarse-grained $(u,v,w)$ onto a divergence free subspace. This projection step is very similar to what SAM would do inside of its time loop.

# Projection method

Many fluid dynamics solvers also need to ensure the velocity field satistfies a divergence free condition. This is traditionally solved using [Chorin's projection method][1].

Let G and D be discrete approximations to the gradient and aneleastic mass conservation equations. Then, Chorin's method tries to find a scalar $\phi$ so that the projected velocity given by
$$ \mathbf{v}^* = \mathbf{v} - G\phi$$
satisfies the mass equation. If this is true, then we can apply $D$ to both sides use the desired fact that $D  \mathbf{v}^* = 0$, to find that 
$$ D G \phi = D \mathbf{v}.$$
This is basically a Poisson-like equation for $\phi$ which we can solve using an iterative solver.

Therefore, we need to find a $G$ and $D$ that are suitable for our data. Unfortunately, the coarse-grained data is on a somewhat strange grid. The points are all collocated horizontally, but $w$ is staggered vertically, morever we need to satisfy the lateral boundary conditions (no-normal flow in $y$, and periodic in $x$). This suggests that we use the Fourier basis in the x-direction, a cosine/sin basis in the y-direction, and a finite difference equation in the vertical. In practice, it is probably easier to use even/odd extension in the $y$ direction to allow using FFT for both directions. I can just never figure out how to use a DCT properly.

Suppose we have decomposed the data in this basis so that
$$
f(x, y, z_j) = D \mathbf{v} = \sum_{k,m} d_{k,m}^j \exp(2\pi i k x/ L_x) \exp(2 \pi i m y / 2 L_y)
$$

then we can define the gradient as follows in the Fourier basis
\begin{align}
(G_x \phi )_{k,m}^j &= \frac{2\pi i k}{L_x} \phi_{k,m}^j\\
(G_y \phi)_{k,m}^j&= \frac{2\pi i m}{2 L_y} \phi_{k,m}^j\\
(G_z \phi)_{k,m}^j&= \frac{\phi_{k,m}^{j+1}-\phi_{k,m}^j}{\Delta z^j}.\\
\end{align}

An approximation to the continuity equation in this same basis is given by

$$ D  (F, G, H) = \frac{2\pi i k}{L_x} F  + \frac{2\pi i m}{2 L_y} G + \frac{\rho^{j+1/2} H^{j+1/2} - \rho^{j-1/2} H^{j-1/2}}{\rho^j (z^{j+1/2}-z^{j-1/2})}.
$$

This approach allows solving the linear system in the z-direction separately for each wave number combination, which can be done quickly using standard dense solvers. Another approach is to use so-called Rhie-Chow interpolation to find the divergence (not sure about the gradient). This is done by performing the following:

$ u[i] = v[i] + (p[i+1] - p[i-1])/2 $
$u[i+.5] = (u[i] + u[i+1])/2 - (p[i+1] - p[i]) = v... p[i+2]/4 - p[i]/4 + p[i+1]/4 - p[i-1]/4 -p[i+1] +p[i] =
v...   (p[i+2] - 3 p[i+1] + 3 p[i] - p[i-1]) /4
$


[1]: https://en.wikipedia.org/wiki/Projection_method_(fluid_dynamics)